<a href="https://colab.research.google.com/github/golu10/projects/blob/master/twittersentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

data=pd.read_csv('/content/train_E6oV3lV.csv')
print(data.head())

import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation


In [0]:
def remove_pattern(pattern,text):
  rem=[x for x in text.split() if x.startswith(pattern)]
  input_text=[x for x in text.split() if x not in rem]
  return ' '.join(input_text)

data['new_tweet']=data['tweet'].apply(lambda x : remove_pattern('@',x))

nlp=spacy.load('en_core_web_sm')
def lemma(text):
  doc=nlp(text)
  lemmas=[token.lemma_ for token in doc]
  lemmas=[i for i in lemmas if i.isalpha()]
  return ' '.join(lemmas)
data['new_tweet']=data['new_tweet'].apply(lambda x:lemma(x))
data.drop(columns='tweet',axis=1)
Y=data['label']
X=data['new_tweet']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

tfidf_vectorizer=TfidfVectorizer(max_df=0.90,min_df=10,stop_words='english')
X_traindf = tfidf_vectorizer.fit_transform(X)

classifier=MultinomialNB()
classifier.fit(X_traindf,Y)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
test_data=pd.read_csv('/content/test_tweets_anuFYb8.csv')
print(test_data.head())
test_data['test_tweet']=test_data['tweet'].apply(lambda x: remove_pattern('@',x))
test_data['test_tweet']=test_data['test_tweet'].apply(lambda x:lemma(x))
X_testy=test_data['test_tweet']
X_testydf=tfidf_vectorizer.transform(X_testy)
test_pred=classifier.predict(X_testydf)
print(test_pred)

      id                                              tweet
0  31963  #studiolife #aislife #requires #passion #dedic...
1  31964   @user #white #supremacists want everyone to s...
2  31965  safe ways to heal your #acne!!    #altwaystohe...
3  31966  is the hp and the cursed child book up for res...
4  31967    3rd #bihday to my amazing, hilarious #nephew...
[0 0 0 ... 0 0 0]


In [0]:
import collections
collections.Counter(test_pred)
submit=pd.read_csv('/content/sample_submission_gfvA5FD.csv')
submit1=submit.drop(['label'],axis=1)
submit1['label']=test_pred
print(submit1['label'].value_counts())
submit1.to_csv('submission0.csv')

0    16679
1      518
Name: label, dtype: int64
